In [5]:
import os
import xml.etree.ElementTree as ET
import random
import json
from tqdm.auto import tqdm # 진행률 표시

# --- 설정 ---
TMX_FILE_PATH = './en-ko.tmx'  # !! 실제 TMX 파일 경로로 수정하세요 !!
SOURCE_LANG_CODE = 'en'
TARGET_LANG_CODE = 'ko'
TOTAL_DATA_SIZE_TO_SELECT = 15000 # 선택할 총 데이터 개수
RANDOM_SEED = 42
OUTPUT_JSON_PATH = './prepared2_data.json' # 저장될 JSON 파일 경로

# --- TMX 파싱 함수 (이전 코드와 동일) ---
def parse_tmx(tmx_file, src_lang, tgt_lang):
    pairs = []
    print(f"'{tmx_file}' 파싱 시작...")
    try:
        tree = ET.parse(tmx_file)
        root = tree.getroot()
        # tqdm으로 진행률 표시 추가
        for tu in tqdm(root.findall('.//tu'), desc="TMX 파싱 중"):
            source_seg = None
            target_seg = None
            for tuv in tu.findall('tuv'):
                lang_attr = '{http://www.w3.org/XML/1998/namespace}lang'
                lang = tuv.get(lang_attr) if tuv.get(lang_attr) else tuv.get('lang')
                seg_element = tuv.find('seg')
                if seg_element is not None and seg_element.text is not None:
                    segment_text = seg_element.text.strip()
                    if lang == src_lang:
                        source_seg = segment_text
                    elif lang == tgt_lang:
                        target_seg = segment_text
            if source_seg and target_seg:
                # 간단한 필터링 예시 (빈 문자열 제외)
                if source_seg and target_seg:
                    pairs.append({'source': source_seg, 'target': target_seg})
    except ET.ParseError as e:
        print(f"XML 파싱 오류: {e}")
        return None
    except FileNotFoundError:
        print(f"오류: 파일 '{tmx_file}'을(를) 찾을 수 없습니다.")
        return None
    except Exception as e:
        print(f"TMX 파싱 중 예상치 못한 오류 발생: {e}")
        return None
    print(f"파싱 완료. 총 {len(pairs)}개의 문장 쌍 로드.")
    return pairs

# --- 메인 로직 ---
if __name__ == "__main__":
    # 랜덤 시드 고정
    random.seed(RANDOM_SEED)

    # TMX 파일 파싱
    all_sentence_pairs = parse_tmx(TMX_FILE_PATH, SOURCE_LANG_CODE, TARGET_LANG_CODE)

    if all_sentence_pairs is None or not all_sentence_pairs:
        print("데이터를 로드하지 못했습니다. 스크립트를 종료합니다.")
        exit()

    # 데이터 셔플
    print("데이터 셔플 중...")
    random.shuffle(all_sentence_pairs)

    # 필요한 총 데이터 개수만큼 선택
    selected_pairs = []
    if len(all_sentence_pairs) < TOTAL_DATA_SIZE_TO_SELECT:
        print(f"경고: 로드된 데이터({len(all_sentence_pairs)}개)가 요청된 총 데이터 크기({TOTAL_DATA_SIZE_TO_SELECT}개)보다 적습니다.")
        print(f"사용 가능한 모든 데이터를 사용합니다.")
        selected_pairs = all_sentence_pairs
    else:
        print(f"로드된 데이터에서 {TOTAL_DATA_SIZE_TO_SELECT}개를 랜덤하게 선택합니다.")
        selected_pairs = all_sentence_pairs[:TOTAL_DATA_SIZE_TO_SELECT]

    # 선택된 데이터를 JSON 파일로 저장
    print(f"선택된 데이터 {len(selected_pairs)}개를 '{OUTPUT_JSON_PATH}' 파일로 저장 중...")
    try:
        with open(OUTPUT_JSON_PATH, 'w', encoding='utf-8') as f:
            json.dump(selected_pairs, f, ensure_ascii=False, indent=4)
        print("데이터 저장 완료.")
    except IOError as e:
        print(f"파일 저장 중 오류 발생: {e}")
    except Exception as e:
        print(f"데이터 저장 중 예상치 못한 오류 발생: {e}")

'./en-ko.tmx' 파싱 시작...


TMX 파싱 중: 100%|██████████| 389633/389633 [00:00<00:00, 677400.38it/s]


파싱 완료. 총 389632개의 문장 쌍 로드.
데이터 셔플 중...
로드된 데이터에서 15000개를 랜덤하게 선택합니다.
선택된 데이터 15000개를 './prepared2_data.json' 파일로 저장 중...
데이터 저장 완료.


In [7]:
import os
import torch
import json
import random
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)
import evaluate
import numpy as np
from datasets import Dataset, DatasetDict
from tqdm.auto import tqdm

# --- 0. 설정 (Configuration) ---
INPUT_JSON_PATH = './prepared2_data.json'
MODEL_CHECKPOINT = "google/mt5-xl"
TRAIN_SIZE = 10000
VAL_SIZE = 2500
TEST_SIZE = 2500
RANDOM_SEED = 42
OUTPUT_DIR = f"./{MODEL_CHECKPOINT.replace('/', '_')}-plz_file"
TRAIN_BATCH_SIZE = 4
EVAL_BATCH_SIZE = 4
LEARNING_RATE = 2e-5
NUM_EPOCHS = 1
WEIGHT_DECAY = 0.01
GRADIENT_ACCUMULATION_STEPS = 2
SAVE_STEPS = 500
EVAL_STEPS = 500
LOGGING_STEPS = 100

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

# --- 1. 준비된 데이터 로드 및 분할 ---
print(f"'{INPUT_JSON_PATH}'에서 데이터 로드 중...")
try:
    with open(INPUT_JSON_PATH, 'r', encoding='utf-8') as f:
        loaded_pairs = json.load(f)
    print(f"총 {len(loaded_pairs)}개의 문장 쌍 로드 완료.")
except FileNotFoundError:
    print(f"오류: 파일 '{INPUT_JSON_PATH}'을(를) 찾을 수 없습니다. 먼저 데이터 준비 스크립트를 실행하세요.")
    exit()
except json.JSONDecodeError:
    print(f"오류: '{INPUT_JSON_PATH}' 파일이 올바른 JSON 형식이 아닙니다.")
    exit()
except Exception as e:
    print(f"데이터 로드 중 예상치 못한 오류 발생: {e}")
    exit()

total_loaded = len(loaded_pairs)
expected_total = TRAIN_SIZE + VAL_SIZE + TEST_SIZE
if total_loaded < expected_total:
    print(f"경고: 로드된 데이터({total_loaded}개)가 필요한 총 데이터({expected_total}개)보다 적습니다.")
    print("사용 가능한 데이터에 맞춰 분할 크기를 조정합니다.")
    ratio = total_loaded / expected_total
    TRAIN_SIZE = int(TRAIN_SIZE * ratio)
    VAL_SIZE = int(VAL_SIZE * ratio)
    TEST_SIZE = total_loaded - TRAIN_SIZE - VAL_SIZE
    print(f"조정된 크기: 훈련={TRAIN_SIZE}, 검증={VAL_SIZE}, 테스트={TEST_SIZE}")
elif total_loaded > expected_total:
    print(f"경고: 로드된 데이터({total_loaded}개)가 필요한 총 데이터({expected_total}개)보다 많습니다.")
    print(f"{expected_total}개만 사용합니다.")
    loaded_pairs = loaded_pairs[:expected_total]

train_pairs_dict = loaded_pairs[:TRAIN_SIZE]
val_pairs_dict = loaded_pairs[TRAIN_SIZE : TRAIN_SIZE + VAL_SIZE]
test_pairs_dict = loaded_pairs[TRAIN_SIZE + VAL_SIZE :]

print(f"데이터 분할 완료: 훈련 {len(train_pairs_dict)}개, 검증 {len(val_pairs_dict)}개, 테스트 {len(test_pairs_dict)}개")

final_datasets = DatasetDict({
    'train': Dataset.from_list(train_pairs_dict),
    'validation': Dataset.from_list(val_pairs_dict),
    'test': Dataset.from_list(test_pairs_dict)
})
print("Hugging Face DatasetDict 생성 완료.")

# --- 2. 토크나이저 로드 및 최대 길이 계산 ---
print(f"\n'{MODEL_CHECKPOINT}' 토크나이저 로딩 중...")
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
    print("토크나이저 로딩 성공.")
except Exception as e:
    print(f"토크나이저 로딩 오류: {e}")
    exit()

max_source_length = 8
max_target_length = 8

# --- 3. 데이터셋 토큰화 ---
def preprocess_function(examples, model_ckpt): # model_ckpt 인자 추가
    inputs = examples['source']
    targets = examples['target']
    tokenizer_local = AutoTokenizer.from_pretrained(model_ckpt) # model_ckpt 사용
    model_inputs = tokenizer_local(inputs, max_length=max_source_length, truncation=True)
    with tokenizer_local.as_target_tokenizer():
        labels = tokenizer_local(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

print("\n데이터셋 토큰화 중...")
num_cores = os.cpu_count()
print(f"사용 가능한 CPU 코어 수: {num_cores}. 토큰화에 활용합니다.")

tokenized_datasets = final_datasets.map(
    preprocess_function,
    batched=True,
    num_proc=max(1, num_cores // 2),
    remove_columns=final_datasets["train"].column_names,
    fn_kwargs={'model_ckpt': MODEL_CHECKPOINT} # fn_kwargs를 통해 MODEL_CHECKPOINT 전달
)
print("데이터셋 토큰화 완료.")

# --- 5. 모델 로드 ---
print(f"\n'{MODEL_CHECKPOINT}' 사전 훈련된 모델 로딩 중...")
try:
    model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)
    print("모델 로딩 성공.")
except Exception as e:
    print(f"모델 로딩 오류: {e}")
    exit()

# --- 6. 데이터 콜레이터 정의 ---
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
print("\n데이터 콜레이터 준비 완료.")

# --- 7. 평가 지표 정의 (BLEU) ---
try:
    bleu_metric = evaluate.load("sacrebleu")
    print("\n평가 지표 (sacrebleu) 로딩 성공.")
except Exception as e:
    print(f"\nsacrebleu 지표 로딩 실패: {e}. BLEU 점수 계산이 비활성화될 수 있습니다.")
    bleu_metric = None

# compute_metrics 함수는 tokenizer를 전역 범위에서 참조하므로 수정 필요 없음
def compute_metrics(eval_preds):
    if bleu_metric is None: return {}
    preds, labels = eval_preds
    if isinstance(preds, tuple): preds = preds[0]
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    decoded_labels_for_bleu = [[label.strip()] for label in decoded_labels]
    decoded_preds_for_bleu = [pred.strip() for pred in decoded_preds]
    result = bleu_metric.compute(predictions=decoded_preds_for_bleu, references=decoded_labels_for_bleu)
    result = {"bleu": result["score"]}
    prediction_lens = np.sum(preds != tokenizer.pad_token_id, axis=1)
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

# --- 8. 훈련 인자(Arguments) 정의 ---
print("\n훈련 인자 설정 중 (CPU 사용)...")
training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    weight_decay=WEIGHT_DECAY,
    save_total_limit=1,
    num_train_epochs=NUM_EPOCHS,
    predict_with_generate=True,
    logging_dir=f"{OUTPUT_DIR}/logs",
    logging_steps=LOGGING_STEPS,
    eval_steps=EVAL_STEPS,
    eval_strategy="steps",
    fp16=False,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    report_to="tensorboard",
    load_best_model_at_end=True,
    metric_for_best_model="bleu",
    greater_is_better=True,
    no_cuda=True,
)

# --- 9. Trainer 생성 ---
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics if bleu_metric else None,
    tokenizer=tokenizer, # Trainer에는 tokenizer를 전달해야 함
)
print("Trainer 준비 완료.")

# --- 10. 미세 조정 시작 ---
print("\n" + "="*30)
print("미세 조정을 시작합니다 (CPU 사용).")
print(f"총 {NUM_EPOCHS} 에폭 동안 훈련합니다. 시간이 매우 오래 걸릴 수 있습니다.")
print("="*30 + "\n")

try:
    print("훈련 시작...")
    train_result = trainer.train(resume_from_checkpoint=None)
    print("훈련 완료.")

    # --- 11. 최종 모델 저장 ---
    print(f"\n훈련 중 최고 성능 모델이 '{training_args.output_dir}' 내 checkpoint 폴더에 저장되었습니다.")
    print(f"훈련 종료 후 최고 성능 모델이 로드되었습니다.")

    metrics = train_result.metrics
    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    trainer.save_state()

    # --- 12. 훈련 후 최종 평가 ---
    print("\n최고 성능 모델 평가 중 (검증 데이터셋)...")
    eval_metrics = trainer.evaluate(eval_dataset=tokenized_datasets["validation"])
    trainer.log_metrics("eval_validation", eval_metrics)
    trainer.save_metrics("eval_validation", eval_metrics)
    print(f"최고 성능 모델 검증 데이터셋 평가 결과: {eval_metrics}")

    print("\n최고 성능 모델 평가 중 (테스트 데이터셋)...")
    test_metrics = trainer.evaluate(eval_dataset=tokenized_datasets["test"])
    trainer.log_metrics("eval_test", test_metrics)
    trainer.save_metrics("eval_test", test_metrics)
    print(f"최고 성능 모델 테스트 데이터셋 평가 결과: {test_metrics}")

except Exception as e:
    print(f"\n훈련 또는 평가 중 오류 발생: {e}")
    import traceback
    traceback.print_exc()

print("\n스크립트 실행 완료.")

'./prepared2_data.json'에서 데이터 로드 중...
총 15000개의 문장 쌍 로드 완료.
데이터 분할 완료: 훈련 10000개, 검증 2500개, 테스트 2500개
Hugging Face DatasetDict 생성 완료.

'google/mt5-xl' 토크나이저 로딩 중...


c:\Users\KDP-34\anaconda3\envs\new_NLP\Lib\site-packages\transformers\convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


토크나이저 로딩 성공.

데이터셋 토큰화 중...
사용 가능한 CPU 코어 수: 8. 토큰화에 활용합니다.


Map (num_proc=4):   0%|          | 0/10000 [00:02<?, ? examples/s]


NameError: name 'AutoTokenizer' is not defined